In [1]:
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import streamlit as st
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import re

In [2]:
# Tipos de usuarios:

Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
Tipo2= "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
Tipo3= "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/"
Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"

In [3]:
def concat(*args):
        strs = [str(arg) for arg in args if not pd.isnull(arg)]
        return ','.join(strs) if strs else np.nan

In [4]:
def FlexicarScraping(tipo):
    #URL=f'https://www.flexicar.es/coches-segunda-mano/#precio_to/{presupuesto}/'
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("start-maximized")
    options.add_argument("--disable-infobars")
    driver = webdriver.Chrome(executable_path=r'/usr/bin/chromedriver',options=options) 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    driver.get(tipo)
    precio= driver.find_elements_by_xpath('//*[@id="listadocoches"]')
    cochesBuenos= [e.text for e in precio]
    for e in cochesBuenos:
        ok=e.split("\n")
    droplist=['Previous','Next',"IVA deducible"]
    cleanList=[e for e in ok if e not in droplist]
    CleanList=[]
    for e in cleanList:
        if not "desde" in e:
             CleanList.append(e)
    priceWithDiscount=CleanList[1::4]
    model=CleanList[2::4]
    cleanColumn=CleanList[3::4]
    for e in cleanColumn:
        ok=e.split("\n")
    cleanColumn2=[e.split("\n") for e in cleanColumn]
    ok2=[]
    for e in cleanColumn2:
        for i in e:
            ok2.append(i.split())
    Año=[e[0] for e in ok2]
    Combustible=[e[2] for e in ok2]
    combustiblesucio=[re.findall(r"([$D-H](\w+))",e) for e in Combustible]
    combustiblesucio=[list(item) for sublist in combustiblesucio for item in sublist]
    combustible=[item for sublist in combustiblesucio for item in sublist]
    gas=[]
    for e in combustible:
        if e == 'Hibrido':
            gas.append(e)
        elif e == "Diesel":
            gas.append(e)
        elif e == "Gasolina":
            gas.append(e)
        elif e == "Electrico":
            gas.append(e)
        elif e == "GLP":
            gas.append(e)
    df = pd.DataFrame({
                   'priceWithDiscount': priceWithDiscount,
                     'Model' : model,
                     'Combustible': gas,
                     'Año': Año,
                     'cleanToSplit': cleanColumn
                  })
    df1 = pd.DataFrame(df.cleanToSplit.str.split(' ',4).tolist(),
                                 columns = ["Año1","KM","3","Ciudad","nan"])
    result = pd.concat([df, df1], axis=1, sort=False)
    flexicar = result.drop(["cleanToSplit",'Año1','3'], axis=1)
    np_concat = np.vectorize(concat)
    flexicar['Ciudad'] = np_concat(flexicar['Ciudad'], flexicar['nan'])
    flexicar = flexicar.drop(['nan'], axis=1)
    
    
    return flexicar.sort_values(by='Año', ascending=[True])

In [5]:
Tipo1=FlexicarScraping(Tipo1)
Tipo2=FlexicarScraping(Tipo2)
Tipo3=FlexicarScraping(Tipo3)
Tipo4=FlexicarScraping(Tipo4)
Tipo5=FlexicarScraping(Tipo5)
Tipo6=FlexicarScraping(Tipo6)
Tipo7=FlexicarScraping(Tipo7)

In [6]:
Tipo2


,priceWithDiscount,Model,Combustible,Año,KM,Ciudad
0,4.990€,Chevrolet Cruze 1.6 16V LS,Gasolina,2010,79.990,"Móstoles,2"
6,12.790€,Mini Countryman 1.6 Cooper S,Gasolina,2011,59.000,Alicante
4,4.990€,Citroen C3 1.1i Attraction,Gasolina,2012,130.000,Alicante
3,6.490€,Fiat 500 1.2 8v 69 CV Lounge EU6,Gasolina,2014,75.000,Vaciamadrid
5,7.690€,Renault Clio Expression 1.2 16v 75,Gasolina,2015,69.990,"Móstoles,2"
1,9.490€,Renault Clio Zen Energy TCe 66kW (90CV),Gasolina,2017,49.990,"Móstoles,2"
2,7.990€,Fiat 500 1.2 8v 51kW (69CV) Lounge,Gasolina,2017,49.990,Vaciamadrid
8,40.990€,Audi A5 sport 2.0 TFSI 140kW S tronic Sportback,Gasolina,2017,61.677,"Rivas,II"
7,15.490€,Volkswagen Polo Sport 1.0 TSI 85kW (115CV) DSG,Gasolina,2019,15.949,Bilbao


In [20]:
Tipo2['URL'] = "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
Tipo2

,priceWithDiscount,Model,Combustible,Año,KM,Ciudad,URL
0,4.990€,Chevrolet Cruze 1.6 16V LS,Gasolina,2010,79.990,"Móstoles,2",https://www.flexicar.es/combustible-gasolina/c...
6,12.790€,Mini Countryman 1.6 Cooper S,Gasolina,2011,59.000,Alicante,https://www.flexicar.es/combustible-gasolina/c...
4,4.990€,Citroen C3 1.1i Attraction,Gasolina,2012,130.000,Alicante,https://www.flexicar.es/combustible-gasolina/c...
3,6.490€,Fiat 500 1.2 8v 69 CV Lounge EU6,Gasolina,2014,75.000,Vaciamadrid,https://www.flexicar.es/combustible-gasolina/c...
5,7.690€,Renault Clio Expression 1.2 16v 75,Gasolina,2015,69.990,"Móstoles,2",https://www.flexicar.es/combustible-gasolina/c...
1,9.490€,Renault Clio Zen Energy TCe 66kW (90CV),Gasolina,2017,49.990,"Móstoles,2",https://www.flexicar.es/combustible-gasolina/c...
2,7.990€,Fiat 500 1.2 8v 51kW (69CV) Lounge,Gasolina,2017,49.990,Vaciamadrid,https://www.flexicar.es/combustible-gasolina/c...
8,40.990€,Audi A5 sport 2.0 TFSI 140kW S tronic Sportback,Gasolina,2017,61.677,"Rivas,II",https://www.flexicar.es/combustible-gasolina/c...
7,15.490€,Volkswagen Polo Sport 1.0 TSI 85kW (115CV) DSG,Gasolina,2019,15.949,Bilbao,https://www.flexicar.es/combustible-gasolina/c...


In [7]:
st.title('¿CUÁL ES EL MEJOR VEHÍCULO QUE SE ADAPTA MEJOR A MIS NECESIDADES?')
km = st.radio("¿Cuántos Km haces al año?", ("Menos de 15.000 km","----"))
km1 = st.selectbox("", ["Más de 15.000 km","----"])
uso = st.selectbox(
    "¿Cómo serán tus desplazamientos con tu vehículo?",
     ["Sobretodo por ciudad y muy pocos viajes por carretera al año.",
     "----"]
)
uso2 = st.selectbox(
    "",
     [
     "Por ciudad y carretera por igual.", "----"]
)
uso3 = st.selectbox(
    "",
     [ "Uso el vehículo sobretodo para viajar y muy poco por ciudad.", "----"
     ]
)

familia = st.selectbox(
    "¿Cuántas personas suelen viajar contigo?",
     ["Suelo viajar sólo y/o una persona más.",
     "----"])

familia2 = st.selectbox(
    "",
     [
     "Suelo viajar con la familia (3 personas máx).", 
     "----"])
familia3 = st.selectbox(
    "",
     ["Suelo viajar con la familia (más de 3 personas).","----"])          



In [10]:
km

'Menos de 15.000 km'

In [8]:
tipo1=["Menos de 15.000 km",
       "Sobretodo por ciudad y muy pocos viajes por carretera al año.",
       "Suelo viajar sólo y/o una persona más."]
tipo2=["Menos de 15.000 km",
       "Por ciudad y carretera por igual.",
       "Suelo viajar sólo y/o una persona más."]
tipo3=["Más de 15.000 km",
       "Uso el vehículo sobretodo para viajar y muy poco por ciudad.",
       "Suelo viajar sólo y/o una persona más."]
tipo4=["Más de 15.000 km",
       "Uso el vehículo sobretodo para viajar y muy poco por ciudad.",
       "Suelo viajar con la familia (más de 3 personas)."]
tipo5=["Menos de 15.000 km",
       "Sobretodo por ciudad y muy pocos viajes por carretera al año.",
       "Suelo viajar con la familia (3 personas máx)"]
tipo6=["Menos de 15.000 km",
       "Sobretodo por ciudad y muy pocos viajes por carretera al año.",
       "Suelo viajar con la familia (más de 3 personas)."]
tipo7=["Más de 15.000 km",
       "Sobretodo por ciudad y muy pocos viajes por carretera al año.",
       "Uso el vehículo sobretodo para viajar y muy poco por ciudad."]

In [9]:
if km  and uso  and familia:
    df=FlexicarScraping(Tipo1)
    st.dataframe(df)
    print("ok")
elif km and uso2  and familia:
    df1=FlexicarScraping(Tipo2)
    st.dataframe(df1)
    print("ok")
elif km1 and uso3  and familia:
    df2=FlexicarScraping(Tipo3)
    st.dataframe(df2)
elif km1 and uso3 and familia2:
    df3=FlexicarScraping(Tipo4)
    st.dataframe(df3)
elif km and uso and familia2:
    df4=FlexicarScraping(Tipo5)
    st.dataframe(df4)
elif km  and uso and familia3:
    df5=FlexicarScraping(Tipo6)
    st.dataframe(df5)
elif km1  and uso3 and familia3:
    df6=FlexicarScraping(Tipo7)
    st.dataframe(df6)

TypeError: Object of type 'DataFrame' is not JSON serializable

In [ ]:
Tipo3


In [ ]:
Tipos= {'Tipo1': "https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/",
        'Tipo2': "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/",
        'Tipo3': "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/",
        'Tipo4': "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/",
        'Tipo5': "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/",
        'Tipo6': "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/",
        'Tipo7': "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/",
       }

# Argumentos

## Tipo de usuarios:

   ## Usuario Tipo 1: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 80% y 20% Extra-Urbanas.
   ### Sobretodo por ciudad y muy pocos viajes por carretera al año.
   ### Conviviendo sólo o con un miembro más.
   
  Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
  
  ## Usuario Tipo 2: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 60% y 40% Extra-Urbanas.
   ### Conviviendo sólo o con un miembro más.
   
  Tipo2="https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
  
   ## Usuario Tipo 3: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Conviviendo sólo con un miembro más.
  Tipo3= 'https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/'
  
   ## Usuario Tipo 4: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Familia con 1 o 2 hijos.
   Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
   
   ## Usuario Tipo 5: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 70% y 30% Extra-Urbanas.
   ### Familia con 1 o 2 hijos.
   Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
   
   ## Usuario Tipo 6: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 60% y 40% Extra-Urbanas.
   ### Familia con más de 2 hijos.
   Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
   
   ## Usuario Tipo 7: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Familia con más de 2 hijos.
   Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"
   